In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models, datasets
from torch.utils.data import DataLoader, random_split
from torchvision.models import ResNet50_Weights

import os
from tqdm import tqdm

In [ ]:
# ✅ Set Paths
DATASET_PATH = r"./data/leaf_coffe_dataset"
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 3

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)
# ✅ Define Image Transformations
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ✅ Load Dataset
dataset = datasets.ImageFolder(DATASET_PATH, transform=transform)

# ✅ Split Dataset: 70% Train, 20% Test, 10% Validation
train_size = int(0.7 * len(dataset))
test_size = int(0.2 * len(dataset))
val_size = len(dataset) - train_size - test_size

train_ds, test_ds, val_ds = random_split(dataset, [train_size, test_size, val_size])

# ✅ Create DataLoaders
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

print(f"Train: {len(train_ds)}, Test: {len(test_ds)}, Validation: {len(val_ds)}")

# ✅ Load Pretrained ResNet50
model = models.resnet50(weights= ResNet50_Weights.DEFAULT)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(dataset.classes))  # Adjust output layer

model = model.to(DEVICE)

# ✅ Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# ✅ Training Loop
def train(model, train_loader, val_loader, epochs):
    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0

        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=True)
        for images, labels in loop:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            loop.set_postfix(loss=loss.item(), acc=correct/total)  # Show live metrics
        
        val_acc = evaluate(model, val_loader)
        print(f"Epoch {epoch+1}/{epochs} | Train Acc: {correct/total:.4f} | Val Acc: {val_acc:.4f}")


# ✅ Evaluation Function
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return correct / total

# ✅ Start Training
train(model, train_loader, val_loader, EPOCHS)

# ✅ Evaluate on Test Data
test_acc = evaluate(model, test_loader)
print(f"Test Accuracy: {test_acc:.4f}")


cuda
Train: 40984, Test: 11709, Validation: 5856


Epoch 1/3: 100%|██████████| 1281/1281 [27:18<00:00,  1.28s/it, acc=0.992, loss=0.219]   


Epoch 1/3 | Train Acc: 0.9923 | Val Acc: 1.0000


Epoch 2/3: 100%|██████████| 1281/1281 [25:18<00:00,  1.19s/it, acc=0.999, loss=0.00062] 


Epoch 2/3 | Train Acc: 0.9990 | Val Acc: 1.0000


Epoch 3/3: 100%|██████████| 1281/1281 [25:30<00:00,  1.19s/it, acc=1, loss=0.00151]     


Epoch 3/3 | Train Acc: 0.9995 | Val Acc: 1.0000
Test Accuracy: 1.0000


In [6]:
torch.save(model.state_dict(), "trained_model.pth")
print("✅ Model weights saved successfully!")


✅ Model weights saved successfully!


In [4]:
torch.save(model, "trained_model_full.pth")
print("✅ Full model saved successfully!")


✅ Full model saved successfully!
